In [1]:
import sys

sys.path.append('..')

In [2]:
from pathlib import Path
from random import sample

import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact
from PIL import Image
from tqdm import tqdm

from digi_leap.ocr import ocr_label
from digi_leap.ocr_score import score_easyocr, score_tesseract

In [3]:
DATA_DIR = Path('..') / 'data'
LABELS_DIR = DATA_DIR / 'labels' / 'typewritten'
# LABELS_DIR = DATA_DIR / 'labels' / 'handwritten'

In [4]:
IMAGES = sorted(LABELS_DIR.glob('*.jpg'))
SAMPLE = sample(IMAGES, 100)

In [5]:
easy_scores = []
tess_scores = []

for path in tqdm(SAMPLE):
    image = Image.open(path)

    tess_score = ocr_label(path, score_tesseract)
    easy_score = ocr_label(path, score_easyocr)

    tess_scores.append(tess_score)
    easy_scores.append(easy_score)

100%|██████████| 100/100 [15:45<00:00,  9.46s/it]


In [6]:
tess_df = pd.DataFrame(tess_scores)
# tess_df.to_csv('../output/tess_scores.csv', index=False)

easy_df = pd.DataFrame(easy_scores)
# easy_df.to_csv('../output/easy_scores.csv', index=False)

In [10]:
df = tess_df.merge(
    easy_df,
    on=['file', 'stem'],
    suffixes=['_tess', '_easy']
)
df = df.reindex(columns=[
    'file', 'stem',
    'total_tess', 'total_easy',
    'found_tess', 'found_easy',
    'method_tess', 'method_easy',
    'text_tess', 'text_easy',
])
df.to_csv('../output/compare_scores.csv', index=False)